In [1]:
#start
import gspread #type:ignore NOQA
import imaplib
import email
import json
from decimal import Decimal

#connecting to GSeets and selecting the file
sa = gspread.service_account(filename="sa_creds.json")

#read credentials from json
with open("ym_creds.json", "r") as f:
    jmCreds = json.load(f)

groceryBrandDoneEmails = [] #list to write to GSheets
def groceryBrandEmails(GroceryBrand: str, emailList):
    for i in emailList:
        f = [GroceryBrand]
        f.append(i)
        groceryBrandDoneEmails.append(f)

sh = sa.open("Grocery")
#selecting sheets:
wksEmails = sh.worksheet("Emails")
wksSummaryMaxima = sh.worksheet("MaximaSummarized")
wksItemsMaxima = sh.worksheet("MaximaItems")

#getting data from wksEmails sheet
# wksEmails.get()
maximaAlreadyIn = [i[1] for i in wksEmails.get() if 'Maxima' in i[0]]

In [2]:
# Connect to the Yahoo Mail IMAP server
imap_server = imaplib.IMAP4_SSL("imap.mail.yahoo.com")
# Login to the account
imap_server.login(jmCreds["email"], jmCreds["password"])
# Select the "Inbox" folder
imap_server.select("Inbox")

# Search for all emails: status, email_ids = imap_server.search(None, "ALL")
# Search for all emails from MAXIMA
status, email_ids = imap_server.search(None, "FROM noreply.code.provider@maxima.lt")
maximaEmails = email_ids[0].decode().split()

maximaToAdd = [i for i in maximaEmails if i not in maximaAlreadyIn]

In [3]:
maximaToAdd

['5539', '5555', '5565']

In [4]:
#Receipt Summary Data:
receiptSummaryDataFULL = []
receiptSummaryData = []
def readMaximaReceiptSummary(EmailId: str, receipt):
    receiptSummaryData.clear()
    for i in range(0, 11):
        receiptSummaryData.append('')
    receiptSummaryData[0] = EmailId
    for i in receipt:
        #Receipt_id
        if '<pre>Kvitas ' in i:
            receiptSummaryData[1] = i.split()[1]
        #BankReceipt_id
        if 'KVITO NR' in i:
            receiptSummaryData[2] = i.split()[2]
        #Receipt_Document#
        if 'DOKUMENTO NR' in i:
            receiptSummaryData[3] = i.split()[2]
        #RRN
        if 'RRN:' in i:
            receiptSummaryData[4] = i.split()[1]
        #Date
        if 'LTF NM ' in i:
            receiptSummaryData[5] = f'{i.split()[3]}-{i.split()[4]}-{i.split()[5]}'
        #Time
        if 'LTF NM ' in i:
            receiptSummaryData[6] = i.split()[6]
        #GroceryBrand
        if 'MAXIMA LT' in i:
            receiptSummaryData[7] = 'MAXIMA LT'
        #Address
        if 'MAXIMA LT, UAB' in i:
            receiptSummaryData[8] = i.split('<br />')[1].split(' Kasa Nr.')[0][:-1]
        #TotalAmount
        if 'Apsipirkimo suma:' in i:
            receiptSummaryData[9] = i.split(': ')[1][:-4]
        #TotalDiscount
        if 'Kvito nuolaidų suma:' in i:
            receiptSummaryData[10] = i.split(': ')[1][:-4]
    receiptSummaryDataFULL.append(list(receiptSummaryData))

#Processed Emails:
processedEmails = []
processedEmail = []
def emailProcessLog(GroceryBrand: str, Email_ids):
    for i in Email_ids:
        processedEmail = [GroceryBrand, i]
        processedEmails.append(list(processedEmail))

#Receipt items data:
items = []
def readMaximaReceiptItems(EmailId: str, receipt):
    
    receiptMainInfo = receipt.decode().split('\r\n')
    for i in receiptMainInfo:
        #Receipt_id
        if '<pre>Kvitas ' in i:
            receiptID = i.split()[1]
        #Date
        if 'LTF NM ' in i:
            receiptDate = f'{i.split()[3]}-{i.split()[4]}-{i.split()[5]}'
        #Time
        if 'LTF NM ' in i:
            receiptTime = i.split()[6]


    receiptDepic = msg2body.decode().split('<pre>Kvitas')
    receiptDepic = receiptDepic[1].split('======================================================')
    receiptDepic = receiptDepic[0].replace(' N\r\n', ' N A\r\n')
    receiptDepic = receiptDepic.replace(' A\r\n','|')
    receiptDepic = receiptDepic.replace('\r\n','|',1)
    receiptDepic = receiptDepic.replace('&#160;',' ')
    receiptDepic = ' '.join(receiptDepic.split())
    receiptDepic = receiptDepic.split('|')
    receiptDepic = receiptDepic[1:len(receiptDepic)-1]

    # item = ['2023-01-01','20:20:20',EmailId,receiptID]
    for i in receiptDepic:
        item = ['2023-01-01','20:20:20',EmailId,receiptID]
        itemSplit = i.split()
        fullPrice = itemSplit[len(itemSplit)-1]

        if 'X' in itemSplit:
            xIndex = itemSplit.index('X')
            theItem = ' '.join(itemSplit[:xIndex-1])
            unitPrice = itemSplit[xIndex-1].replace(',','.')
            quantity = itemSplit[xIndex+1].replace(',','.')
            measure = itemSplit[xIndex+2]
        else:
            theItem = ' '.join(itemSplit[:-1])
            unitPrice = fullPrice.replace(',','.')
            quantity = ''
            measure = ''

        if fullPrice == 'N':
            fullPrice = itemSplit[len(itemSplit)-2].replace(',','.')
            items[-1][-2] = fullPrice
        elif '-' in fullPrice:
            fullPrice = fullPrice[1:].replace(',','.')
            items[-1][9] = fullPrice
            items[-1][10] = f'{str(round(float(fullPrice)/float(items[-1][8])*100,1))}%'
            items[-1][11] = str(round(float(items[-1][8])-float(fullPrice),2))
        else:
            fullPrice = fullPrice.replace(',','.')
            item.append(theItem)
            item.append(unitPrice)
            item.append(quantity)
            item.append(measure)
            item.append(fullPrice)
            item.append('0.00')
            item.append('0.0%')
            item.append(fullPrice)
            item.append('0.00')
            item.append(i)
            #adding item to items
            items.append(list(item))
    


In [5]:
#writing email summary
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])
    
    msg1 = email_message.get_payload()[0]
    msg1body = msg1.get_payload(decode=True)
    singleEmail = msg1body.decode().split('\r\n')
    readMaximaReceiptSummary(i, singleEmail)
wksSummaryMaxima.append_rows(receiptSummaryDataFULL)

#writing processed emails to Emails sheet
emailProcessLog('Maxima',maximaToAdd)
wksEmails.append_rows(processedEmails)


{'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
 'tableRange': 'Emails!A1:B209',
 'updates': {'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
  'updatedRange': 'Emails!A210:B211',
  'updatedRows': 2,
  'updatedColumns': 2,
  'updatedCells': 4}}

In [13]:
# Disconnect from the server
imap_server.close()
imap_server.logout()

('BYE', [b'IMAP4rev1 Server logging out'])

In [8]:
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])

In [9]:
#get message bu ID
status, email_data = imap_server.fetch("5539", "(RFC822)")
email_message = email.message_from_bytes(email_data[0][1])

In [10]:
msg1 = email_message.get_payload()[0]
msg1body = msg1.get_payload(decode=True)
msg2 = email_message.get_payload()[0]
msg2body = msg2.get_payload(decode=True)
msg3 = email_message.get_payload()[0]
msg3body = msg3.get_payload(decode=True)

In [12]:
# print(msg1.get_payload(decode=True), msg2.get_payload(decode=True))
print(msg2body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 15.81 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 8.66 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.16 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

In [13]:
x = msg2body.decode().split('\r\n')
for e, i in enumerate(x):
    print(e, x[e])
# x[10]

0 <html><body>
1 
2 
3 <html>
4 
5 <body>
6 
7     <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
8             <p>Apsipirkimo suma: 15.81 EUR</p>
9             <br />
10                 <p>Kvito nuolaidų suma: 8.66 EUR</p>
11                 <br />
12                 <p> MAXIMOS PINIGŲ likutis: 0.16 EUR </p>
13             <p>
14                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
15                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
16                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
17                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwn

In [15]:
w = msg2body.decode().split('\r\n')
for e, i in enumerate(w):
    print(e, w[e])
# w[10]

0 <html><body>
1 
2 
3 <html>
4 
5 <body>
6 
7     <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
8             <p>Apsipirkimo suma: 15.81 EUR</p>
9             <br />
10                 <p>Kvito nuolaidų suma: 8.66 EUR</p>
11                 <br />
12                 <p> MAXIMOS PINIGŲ likutis: 0.16 EUR </p>
13             <p>
14                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
15                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
16                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
17                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwn

In [16]:
for i in w:
    if 'DOKUMENTO NR' in i:
        print(i)

DOKUMENTO NR       359505-230126182854                 #


In [19]:
q = msg2body.decode().split('<pre>Kvitas')
q = q[1].split('======================================================')
q = q[0].replace(' N\r\n', ' N A\r\n')
q = q.replace(' A\r\n','|')
q = q.replace('\r\n','|',1)
q = q.replace('&#160;',' ')
q = ' '.join(q.split())
q = q.split('|')
q = q[1:len(q)-1]

In [20]:
q

['Vytinta ispaniška dešra FUETEC, I r. 3,29 X 2 vnt. 6,58',
 'Nuolaida:Vytinta ispaniška dešra FUETEC,2 už 3,99 -2,59',
 'Virta UTENOS DAKTARIŠKA dešra, a. r. 2,19',
 'Sausainiai SMĖLIO JUOSTA 3,59',
 'Nuolaida prekei -0,90',
 'Tamsusis šokoladas MILKA su Alpių karvių pienu 1,55 X 2 vnt. 3,10',
 'Nuolaida prekei -1,56',
 'Šokoladas MILKA OREO CHOCO 1,55',
 'Nuolaida prekei -0,78',
 'Pieninis šokoladas MILKA ALPINE MILK 1,55',
 'Nuolaida prekei -0,78',
 'Glaistytas varškės sūrelis KARUMS su vanilinu, 22,5% ri 0,59 X 4 pakel. 2,36',
 'Glaistytas sūrelis MAGIJA su vanile, 24 % rieb. 0,59 X 4 vnt. 2,36',
 'Nuolaida:Glaistytas sūrelis MAGIJA; 1+1 -1,18',
 'Gazuotas gėrimas COCA COLA 1,09',
 'PET (depozitinis) 0,10 EUR 0,10 N']

In [30]:
for e, i in enumerate(q[0:15]):
    # print(i.split())
    for ee, ii in enumerate(i.split()):
        if ii == 'X' and 'vnt.' in i.split():
        # if ii == 'X' and 'kg' in i.split():
        # if ii == 'X':
            print(ee)
            # print(i[6])
            print(i.split()[ee-1], ii, i.split()[ee+1])
    # if i.split()[len(i.split())-1] != 'N':
        # print(float(i.split()[len(i.split())-1].replace(',','.')))

7
1,19 X 5
6
0,85 X 3
5
0,10 X 3


In [32]:
# for i in q:
#     if 'X' in i.split():
#         print(i)

Pienas, 2 % riebumo, be laktozės 1,19 X 5 vnt. 5,95
GUBERNIJOS tamsi ruginė duonos gira 0,85 X 3 vnt. 2,55
Skardinė (depozitinė) 0,10 EUR 0,10 X 3 vnt. 0,30 N
Glaistyti zefyrai 6,49 X 0,374 kg 2,43
Glaistytas sūrelis MAGIJA su vanile, 24 % rieb. 0,59 X 4 vnt. 2,36
Glaistytas VILKYŠKIŲ vanilės sk. varškės sūrelis, 24% rie 0,59 X 3 vnt. 1,77


In [33]:
from decimal import Decimal
price = "8,33"
# price = Decimal(price.replace(',','.'))
# price + 10.12
price = float(price.replace(',','.'))
str(price + 10.22)


'18.55'

In [34]:
item = ['2023-01-01','20:20:20','5555','231/1119']
#getting final price:
for e, i in enumerate(q):
    finalPrice = i.split()[len(i.split())-1]
    if finalPrice == 'N':
        finalPrice = i.split()[len(i.split())-2]
    finalPrice = float(finalPrice.replace(',','.'))
    print(finalPrice)

5.95
2.55
0.3
1.09
-0.33
0.1
1.25
-0.3
0.1
1.09
0.1
1.39
-0.28
2.43
0.35
-0.11
2.36
1.77
0.59
-0.1
3.79
-0.8


In [21]:
#getting list of items:
items = []
# item = ['2023-01-01','20:20:20','5555','231/1119']
for i in q:
    item = ['2023-01-01','20:20:20','5555','231/1119']
    itemSplit = i.split()
    fullPrice = itemSplit[len(itemSplit)-1]

    if 'X' in itemSplit:
        xIndex = itemSplit.index('X')
        theItem = ' '.join(itemSplit[:xIndex-1])
        unitPrice = itemSplit[xIndex-1].replace(',','.')
        quantity = itemSplit[xIndex+1].replace(',','.')
        measure = itemSplit[xIndex+2]
    else:
        theItem = ' '.join(itemSplit[:-1])
        unitPrice = fullPrice.replace(',','.')
        quantity = ''
        measure = ''

    if fullPrice == 'N':
        fullPrice = itemSplit[len(itemSplit)-2].replace(',','.')
        items[-1][-2] = fullPrice
    elif '-' in fullPrice:
        fullPrice = fullPrice[1:].replace(',','.')
        items[-1][9] = fullPrice
        items[-1][10] = f'{str(round(float(fullPrice)/float(items[-1][8])*100,1))}%'
        items[-1][11] = str(round(float(items[-1][8])-float(fullPrice),2))
    else:
        fullPrice = fullPrice.replace(',','.')
        item.append(theItem)
        item.append(unitPrice)
        item.append(quantity)
        item.append(measure)
        item.append(fullPrice)
        item.append('0.00')
        item.append('0.0%')
        item.append(fullPrice)
        item.append('0.00')
        item.append(i)
        #adding item to items
        items.append(list(item))
items

[['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Vytinta ispaniška dešra FUETEC, I r.',
  '3.29',
  '2',
  'vnt.',
  '6.58',
  '2.59',
  '39.4%',
  '3.99',
  '0.00',
  'Vytinta ispaniška dešra FUETEC, I r. 3,29 X 2 vnt. 6,58'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Virta UTENOS DAKTARIŠKA dešra, a. r.',
  '2.19',
  '',
  '',
  '2.19',
  '0.00',
  '0.0%',
  '2.19',
  '0.00',
  'Virta UTENOS DAKTARIŠKA dešra, a. r. 2,19'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Sausainiai SMĖLIO JUOSTA',
  '3.59',
  '',
  '',
  '3.59',
  '0.90',
  '25.1%',
  '2.69',
  '0.00',
  'Sausainiai SMĖLIO JUOSTA 3,59'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Tamsusis šokoladas MILKA su Alpių karvių pienu',
  '1.55',
  '2',
  'vnt.',
  '3.10',
  '1.56',
  '50.3%',
  '1.54',
  '0.00',
  'Tamsusis šokoladas MILKA su Alpių karvių pienu 1,55 X 2 vnt. 3,10'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Šokoladas MILKA OREO CHOCO',
  '1.55',
  '

In [79]:
# items[-1][-5]
items[-1][-3]
# round(float(items[-1][-5])/float(items[-1][-6])*100,2)


'0.0%'

In [14]:
wksItemsMaxima.append_rows(items)

{'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
 'tableRange': 'MaximaItems!A1:N13',
 'updates': {'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
  'updatedRange': 'MaximaItems!A14:N25',
  'updatedRows': 12,
  'updatedColumns': 14,
  'updatedCells': 168}}

In [6]:
items = []
def readMaximaReceiptItems(EmailId: str, receipt):
    
    receiptMainInfo = receipt.decode().split('\r\n')
    for i in receiptMainInfo:
        #Receipt_id
        if '<pre>Kvitas ' in i:
            receiptID = i.split()[1]
        #Date
        if 'LTF NM ' in i:
            receiptDate = f'{i.split()[3]}-{i.split()[4]}-{i.split()[5]}'
        #Time
        if 'LTF NM ' in i:
            receiptTime = i.split()[6]


    receiptDepic = msg2body.decode().split('<pre>Kvitas')
    receiptDepic = receiptDepic[1].split('======================================================')
    receiptDepic = receiptDepic[0].replace(' N\r\n', ' N A\r\n')
    receiptDepic = receiptDepic.replace(' A\r\n','|')
    receiptDepic = receiptDepic.replace('\r\n','|',1)
    receiptDepic = receiptDepic.replace('&#160;',' ')
    receiptDepic = ' '.join(receiptDepic.split())
    receiptDepic = receiptDepic.split('|')
    receiptDepic = receiptDepic[1:len(receiptDepic)-1]

    # item = ['2023-01-01','20:20:20',EmailId,receiptID]
    for i in receiptDepic:
        item = ['2023-01-01','20:20:20',EmailId,receiptID]
        itemSplit = i.split()
        fullPrice = itemSplit[len(itemSplit)-1]

        if 'X' in itemSplit:
            xIndex = itemSplit.index('X')
            theItem = ' '.join(itemSplit[:xIndex-1])
            unitPrice = itemSplit[xIndex-1].replace(',','.')
            quantity = itemSplit[xIndex+1].replace(',','.')
            measure = itemSplit[xIndex+2]
        else:
            theItem = ' '.join(itemSplit[:-1])
            unitPrice = fullPrice.replace(',','.')
            quantity = ''
            measure = ''

        if fullPrice == 'N':
            fullPrice = itemSplit[len(itemSplit)-2].replace(',','.')
            items[-1][-2] = fullPrice
        elif '-' in fullPrice:
            fullPrice = fullPrice[1:].replace(',','.')
            items[-1][9] = fullPrice
            items[-1][10] = f'{str(round(float(fullPrice)/float(items[-1][8])*100,1))}%'
            items[-1][11] = str(round(float(items[-1][8])-float(fullPrice),2))
        else:
            fullPrice = fullPrice.replace(',','.')
            item.append(theItem)
            item.append(unitPrice)
            item.append(quantity)
            item.append(measure)
            item.append(fullPrice)
            item.append('0.00')
            item.append('0.0%')
            item.append(fullPrice)
            item.append('0.00')
            item.append(i)
            #adding item to items
            items.append(list(item))
    

In [11]:
readMaximaReceiptItems('1234', msg1body)

In [12]:
items

[['2023-01-01',
  '20:20:20',
  '1234',
  '109/2211',
  'Pienas, 2 % riebumo, be laktozės',
  '1.19',
  '5',
  'vnt.',
  '5.95',
  '0.00',
  '0.0%',
  '5.95',
  '0.00',
  'Pienas, 2 % riebumo, be laktozės 1,19 X 5 vnt. 5,95'],
 ['2023-01-01',
  '20:20:20',
  '1234',
  '109/2211',
  'GUBERNIJOS tamsi ruginė duonos gira',
  '0.85',
  '3',
  'vnt.',
  '2.55',
  '0.00',
  '0.0%',
  '2.55',
  '0.30',
  'GUBERNIJOS tamsi ruginė duonos gira 0,85 X 3 vnt. 2,55'],
 ['2023-01-01',
  '20:20:20',
  '1234',
  '109/2211',
  'VOLFO ENGELMAN nealkoholinis alaus kokteilis KRIEK, 0%',
  '1.09',
  '',
  '',
  '1.09',
  '0.33',
  '30.3%',
  '0.76',
  '0.10',
  'VOLFO ENGELMAN nealkoholinis alaus kokteilis KRIEK, 0% 1,09'],
 ['2023-01-01',
  '20:20:20',
  '1234',
  '109/2211',
  'Nealkoholinis UTENOS alaus kokteilis RADLER (mangų skon',
  '1.25',
  '',
  '',
  '1.25',
  '0.30',
  '24.0%',
  '0.95',
  '0.10',
  'Nealkoholinis UTENOS alaus kokteilis RADLER (mangų skon 1,25'],
 ['2023-01-01',
  '20:20:20',
  